In [3]:
import pickle
import json

import numpy as np
from numpy.linalg import norm
import pandas as pd
import re

from keras.utils import to_categorical

from googletrans import Translator

import datetime
import time

import os

#このpgmの格納フォルダ
this_pgm_folder = os.getcwd()

In [7]:
#pickleでファイル保存
#引数：保存対象オブジェクト、ファイル名(拡張子なし)
def save_as_pickle(obj , fname):
    fname_with_extension = fname + '.txt'
    fw = open(fname_with_extension , 'wb')
    pickle.dump(obj , fw)
    fw.close()
    #files.download(fname_with_extension)

#pickleデータを取得
#引数：対象ファイルのパス
def import_pickle(fpass):
    fr = open(fpass , 'rb')
    f = pickle.load(fr)
    fr.close()
    return f

#◆jsonファイルで保存
#引数：保存対象オブジェクト、ファイル名(拡張子なし)
def save_as_json(obj , fname):
    json_fname_with_extension = fname + '.json'
    fw = open(json_fname_with_extension , 'w')
    json.dump(obj , fw , indent = 4)
    fw.close()
    #files.download(json_fname_with_extension)

#◆jsonファイルを取得
#引数：対象ファイルのパス
def import_json(fpass):
    fr = open(fpass , 'r')
    f = json.load(fr)
    fr.close()
    return f




#各単語の分散表現取得
#引数：単語toIDの辞書、重み行列
#戻り値；単語to分散表現のリスト
def get_distributed_representation(word2id , weights):
    word2disvec = {}
    for word , wid in word2id.items():
        #one-hot化
        oh_vec = to_categorical(wid , num_classes = len(word2id) + 1)
        #分散表現
        word2disvec[word] = np.dot(oh_vec , weights)
    
    return word2disvec
    
    
#関連単語top（単語が引数）
#引数：対象単語、単語to分散表現のリスト（get_distributed_representationの戻り値）、出力する関連単語上位 n 
def related_word_top(given_word , word2disvec , top_num):
    #対象単語の分散表現取得
    given_disvec = word2disvec[given_word]
    
    #キー：単語、値：コサイン類似度　の辞書。（対象単語以外）
    dict_cos_degree = {}
    for word , disvec in word2disvec.items():
        if word == given_word:
            continue
        else:
            cos_degree = np.dot(given_disvec , disvec) / (norm(given_disvec) * norm(disvec))
            dict_cos_degree[word] = cos_degree
            
            #print('word' , word , 'cos_degree' ,  cos_degree)
    
    #リスト型
    return sorted(dict_cos_degree.items() , key = lambda x : -x[1])[:top_num]
            
#関連単語top（ベクトルが引数）
#引数：対象ベクトル、単語to分散表現のリスト（get_distributed_representationの戻り値）、出力する関連単語上位 n 
def related_word_top_for_given_disvec(given_disvec , word2disvec , top_num):
    #対象単語の分散表現取得
    #given_disvec = word2disvec[given_word]
    
    #キー：単語、値：コサイン類似度　の辞書。（対象単語以外）
    dict_cos_degree = {}
    for word , disvec in word2disvec.items():
        if disvec is given_disvec:
            continue
        else:
            cos_degree = np.dot(given_disvec , disvec) / (norm(given_disvec) * norm(disvec))
            dict_cos_degree[word] = cos_degree
            
            #print('word' , word , 'cos_degree' ,  cos_degree)
    
    #リスト型
    return sorted(dict_cos_degree.items() , key = lambda x : -x[1])[:top_num]     


In [8]:
#重み行列
wights_pass = os.path.join(this_pgm_folder , 'model_etc' , 'weights.txt')
weights = import_pickle(wights_pass)

#単語2id、id2単語
word2id_pass = os.path.join(this_pgm_folder , 'model_etc' , 'word2id.json')
id2word_pass = os.path.join(this_pgm_folder , 'model_etc' , 'id2word.json')
word2id = import_json(word2id_pass)
id2word = import_json(id2word_pass)

#単語2分散表現(ローカルだと処理に時間かかるのでファイルimport)
#start = datetime.datetime.now()

word2disvec_pass = os.path.join(this_pgm_folder , 'model_etc' , 'word2disvec.txt')
word2disvec = import_pickle(word2disvec_pass)
#word2disvec = get_distributed_representation(word2id , weights)

#end = datetime.datetime.now()
#print('word2disvec_processing_time :' , end - start)

#単語リスト（昇順）＋連番のリスト（ユーザー選択時使用）、その連番と分散表現の辞書
words_list = list(zip(sorted([word for word , id_ in word2id.items()]) , [i for i in range(len(word2id))]))
id2disvec_wordslist = { id_ : word2disvec[word] for word , id_  in words_list}

In [4]:
#pass_ = r'C:\Users\Takayoshi_Hoshino\Desktop\フリーランス\ポートフォリオ\AI\言語処理\text8(wiki)\【for_apllication_pgm】model_etc\word2disvec'
#save_as_pickle(word2disvec , pass_)

In [4]:
#words_list_pass = r'C:\Users\Takayoshi_Hoshino\Desktop\フリーランス\ポートフォリオ\AI\言語処理\text8(wiki)\【for_apllication_pgm】model_etc\words_list.xlsx'
#pd.DataFrame(words_list , columns = ['word' , 'number']).to_excel(words_list_pass)

In [9]:
#戻り値：'continue'（継続する場合）、'end'（終了する場合）
def main_1_calc():
    continue_ = 'continue'
    end = 'end'
    
    #演算（第一引数　ー　第二引数　or 第一引数　＋　第二引数）※演算子は第三引数で指定
    #引数：単語連番（words_listの番号）、単語連番（words_listの番号）
    def calc_vec(vec_1 , vec_2 , operation):
        #足し算
        if operation == '+':
            return vec_1 + vec_2
        elif operation == '-':
            return vec_1 - vec_2
        
    #単語を表示して、演算式を表示（確認用）
    #引数：要素リスト、演算子リスト
    def print_calc_words(calc_entry_decomp , calc_operation_decomp):
        words = list(map(lambda id_ : list(filter(lambda x : str(x[1]) == str(id_) , words_list))[0][0] , calc_entry_decomp))
        operation = calc_operation_decomp + ['']
        zip_list = list(zip(words , operation))
        
        arithmeric_expression_list = []
        arithmeric_expression_list.append(zip_list[0][0])        
        arithmeric_expression_list = arithmeric_expression_list + [str(zip_list[i][1]) + ' ' + str(zip_list[i + 1][0]) for i in range(len(zip_list) - 1)]
        return ' '.join(arithmeric_expression_list)
    
    print('単語IDを使って、演算式を入力してください。(例：「1 + 2 - 3」)')
    input_calc_str = input('演算式（終了する場合は「0」）：')
    
    #終了する場合
    if str(input_calc_str) == '0':
        return end
    
    #演算式の要素をリスト化
    calc_entry_decomp_tmp = re.sub(r'\D' , '@' , input_calc_str.replace(' ' , '')).replace(' ' , '')
    calc_entry_decomp = [x for x in calc_entry_decomp_tmp.split('@')]
    
    #演算式の演算子をリスト化
    calc_operation_decomp_tmp = re.sub(r'\d' , '' , input_calc_str).replace(' ' , '')
    calc_operation_decomp = [x for x in calc_operation_decomp_tmp]
    
    #要素数 = 演算子 + 1 かを確認
    if len(calc_entry_decomp) != len(calc_operation_decomp) + 1:
        print('式が正しくありません。再度入力してください。')
        print()
        return continue_
    
    print_calc_words_ = print_calc_words(calc_entry_decomp , calc_operation_decomp)
    print('演算式：' , print_calc_words_)
    #okならば y 、入力し直す場合は n
    expession_ok_flg = input('上記の式でよろしいでしょうか？ y/n：')
    if expession_ok_flg != 'y':
        print('式が正しくありません。再度入力してください。')
        print()
        return continue_ 
    
    #演算処理
    #zip化、終了判定のため、追加
    calc_result_vec = None
    
    if len(calc_operation_decomp) == 1:
        calc_result_vec = id2disvec_wordslist[int(calc_entry_decomp[0])]
    else:
        for i in range(len(calc_entry_decomp) - 1):        
            vec_1 = id2disvec_wordslist[int(calc_entry_decomp[i])] if i == 0 else calc_result_vec
            vec_2 = id2disvec_wordslist[int(calc_entry_decomp[i + 1])]
            operation = calc_operation_decomp[i]
            calc_result_vec = calc_vec(vec_1 , vec_2 , operation)
    
    #出力する上位関連単語数
    top_num = int(input('表示する関連単語数を入力してください。(1~' + str(len(word2disvec)) + ')：'))
    
    #関連単語リスト作成、結果出力
    print('...')
    top_related_words_list = related_word_top_for_given_disvec(calc_result_vec , word2disvec , top_num)
    
    for i in range(len(top_related_words_list)):
        word = top_related_words_list[i][0]
        degree_of_similarity = top_related_words_list[i][1]
        
        print('{0}位： {1} (類似度：{2}%)'.format(i+1 , word , round(degree_of_similarity * 100 , 2)))
        
    #継続確認
    continue_flg = input('演算を続けますか？ y/n：')
    if continue_flg == 'y':
        return continue_
    else:
        return end
    

In [10]:
#king 34872
#man 38724
#woman 67978

#メイン処理
print('------start------')

continue_flg = ''
while(continue_flg != 'end'):
    continue_flg = main_1_calc()

print('------end------')

------start------
単語IDを使って、演算式を入力してください。(例：「1 + 2 - 3」)
演算式（終了する場合は「0」）：34872 - 38724 + 67978 + 67978
演算式： king - man + woman + woman
上記の式でよろしいでしょうか？ y/n：y
表示する関連単語数を入力してください。(1~69258)：10
...
1位： woman (類似度：89.25%)
2位： palaeologus (類似度：85.89%)
3位： crushes (類似度：85.76%)
4位： infanta (類似度：84.98%)
5位： holwg (類似度：84.8%)
6位： memel (類似度：84.71%)
7位： fountainheads (類似度：84.68%)
8位： torgau (類似度：84.63%)
9位： septimius (類似度：84.59%)
10位： involvements (類似度：84.56%)
演算を続けますか？ y/n：n
------end------


In [ ]:
#beatles 7117
#england 21422
#america 2682
#japan 33162
#genius 25795
#math 39490
#psychology 50616
#adler 915